In [ ]:
# jupyter_vim

In [1]:
# Run Script below
! python /home/app/kafka/src/create_topic.py --topic=book --file=/home/app/data/brothers-karamazov.txt --format=json

<BrokerConnection node_id=bootstrap-0 host=kafka-broker:9092 <connecting> [IPv4 ('172.23.0.4', 9092)]>: connecting to kafka-broker:9092 [('172.23.0.4', 9092) IPv4]
Probing node bootstrap-0 broker version
<BrokerConnection node_id=bootstrap-0 host=kafka-broker:9092 <connecting> [IPv4 ('172.23.0.4', 9092)]>: Connection complete.
Broker version identified as 2.6.0
Set configuration api_version=(2, 6, 0) to skip auto check_version requests on startup
Probing node bootstrap-0 broker version
Broker version identified as 2.6.0
Set configuration api_version=(2, 6, 0) to skip auto check_version requests on startup
<BrokerConnection node_id=1 host=kafka-broker:9092 <connecting> [IPv4 ('172.23.0.4', 9092)]>: connecting to kafka-broker:9092 [('172.23.0.4', 9092) IPv4]
Probing node 1 broker version
<BrokerConnection node_id=1 host=kafka-broker:9092 <connecting> [IPv4 ('172.23.0.4', 9092)]>: Connection complete.
<BrokerConnection node_id=bootstrap-0 host=kafka-broker:9092 <connected> [IPv4 ('172.23.

In [18]:

! export PYTHONPATH=../risingwave:$PYTHONPATH && python ../risingwave/src/main.py --topic=book --bootstrap-servers=kafka-broker:9092 --schema='{"line": "INTEGER","text": "VARCHAR"}' --source='kafka'



                    CREATE SOURCE IF NOT EXISTS "book" (
                        line INTEGER, text VARCHAR
                        )
                    INCLUDE header AS kafka_header,
                    INCLUDE timestamp AS kafka_timestamp
                    WITH (
                        connector = 'kafka',
                        topic='book',
                        properties.bootstrap.server='kafka-broker:9092',
                        scan.startup.mode='earliest',
                        properties.client.id='risingwave',
                        properties.fetch.queue.backoff.ms=1000,
                        properties.statistics.interval.ms=1000
                    ) FORMAT PLAIN ENCODE JSON;
                    


In [19]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(host="risingwave", port=4566, user="root", dbname="dev")
conn.autocommit = True

In [20]:
with conn.cursor() as cur:
    cur.execute("SELECT kafka_timestamp, line, text FROM book LIMIT 10;")
    values = cur.fetchall()

In [21]:
pd.DataFrame(values, columns=["timestamp", "line", "text"])

,timestamp,line,text
0,2025-03-11 02:14:32.514000+00:00,1,The Project Gutenberg eBook of The Brothers Ka...
1,2025-03-11 02:14:32.514000+00:00,2,
2,2025-03-11 02:14:32.514000+00:00,3,This ebook is for the use of anyone anywhere i...
3,2025-03-11 02:14:32.514000+00:00,4,most other parts of the world at no cost and w...
4,2025-03-11 02:14:32.514000+00:00,5,"whatsoever. You may copy it, give it away or r..."
5,2025-03-11 02:14:32.514000+00:00,6,of the Project Gutenberg License included with...
6,2025-03-11 02:14:32.514000+00:00,7,at www.gutenberg.org. If you are not located i...
7,2025-03-11 02:14:32.514000+00:00,8,you will have to check the laws of the country...
8,2025-03-11 02:14:32.514000+00:00,9,before using this eBook.
9,2025-03-11 02:14:32.514000+00:00,10,


In [29]:
# create materialized view
with conn.cursor() as cur:
    cur.execute("DROP MATERIALIZED VIEW IF EXISTS book_view")
    cur.execute(
        """
        CREATE MATERIALIZED VIEW book_view AS
        SELECT
            date_trunc('second', kafka_timestamp) AS window_start,
            SUM(LENGTH(text)) AS total_characters,
            count(*) AS total_lines
        FROM
            book
        GROUP BY
            window_start;
        """)

In [30]:
with conn.cursor() as cur:
    cur.execute("SELECT * FROM book_view;")
    values = cur.fetchall()
pd.DataFrame(values, columns=['timestamp', 'total_characters', 'total_lines'])
    

ValueError: 2 columns passed, passed data had 3 columns

In [ ]:
import duckdb

con = duckdb.connect()
con.sql(
    f"""
    INSTALL postgres;
    LOAD postgres;  
    ATTACH 'host=risingwave port=4566 dbname=dev user=root' AS db (TYPE postgres);  
"""
)
con.sql("SHOW DATABASES;")
con.sql("SHOW ALL TABLES;")